In [1]:
import openai


In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
import os

In [ ]:
os.environ

In [ ]:
response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = [{"role": "user", "content":"is it too late to join the course?"}]
)

In [ ]:
response

In [6]:
from huggingface_hub import login

login()  # You will be prompted for your HF key, which will then be saved locally

In [ ]:
# Create Hugging Face client (similar to OpenAI)
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    repetition_penalty=1.03,
)

client = ChatHuggingFace(llm=llm, verbose=False)

In [13]:
# Use the same message format as OpenAI
from langchain_core.messages import HumanMessage, SystemMessage

# Method 1: Using LangChain message objects (recommended)
messages = [
    HumanMessage(content="is it too late to join the course?")
]

# Get response (similar to OpenAI's client.chat.completions.create)
response = client.invoke(messages)

# Access the response content (like OpenAI's response.choices[0].message.content)
print("Response:", response.content)

Response: I'd be happy to help, but I don't have any information about a specific course. Can you please provide more context or details about the course you're interested in? Such as:

* What type of course is it (online, offline, university, etc.)?
* What's the name of the course?
* What's the deadline for joining the course?

This will help me provide you with a more accurate answer about whether it's too late to join the course.


In [15]:
# Create a simple wrapper class to make it work exactly like OpenAI
class HuggingFaceClient:
    def __init__(self, client):
        self.chat = ChatCompletions(client)

class ChatCompletions:
    def __init__(self, client):
        self.client = client
    
    def create(self, model=None, messages=None, **kwargs):
        """
        Mimics OpenAI's chat.completions.create() method
        messages format: [{"role": "user", "content": "your message"}]
        """
        from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
        
        # Convert OpenAI format to LangChain format
        langchain_messages = []
        for msg in messages:
            role = msg.get("role")
            content = msg.get("content")
            
            if role == "system":
                langchain_messages.append(SystemMessage(content=content))
            elif role == "user":
                langchain_messages.append(HumanMessage(content=content))
            elif role == "assistant":
                langchain_messages.append(AIMessage(content=content))
        
        # Get response from Hugging Face
        ai_response = self.client.invoke(langchain_messages)
        
        # Return an object that mimics OpenAI's response structure
        class Response:
            def __init__(self, content):
                self.choices = [type('obj', (object,), {
                    'message': type('obj', (object,), {
                        'content': content
                    })()
                })()]
        
        return Response(ai_response.content)

# Wrap the Hugging Face client
hf_client = HuggingFaceClient(client)

In [16]:
# Now use it EXACTLY like OpenAI API!
response = hf_client.chat.create(
    model='microsoft/Phi-3-mini-4k-instruct',  # model param is optional here
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

# Access response the same way as OpenAI
print("OpenAI-style response:", response.choices[0].message.content)

OpenAI-style response: I'd be happy to help you with that, but I don't have any information about a specific course you're referring to. Could you please provide more context or details about the course you're interested in joining, such as:

* The name of the course
* The type of course (e.g. online, in-person, university, etc.)
* The deadline for joining the course
* Any relevant details about the course (e.g. prerequisites, requirements, etc.)

Once I have this information, I'll do my best to provide you with an answer about whether it's too late to join the course.


In [24]:
response.choices[0].message.content

"I'm happy to help you, but I don't have any information about a specific course you're referring to. Could you please provide more context or details about the course you're interested in joining? That way, I can better understand your situation and provide a more accurate response."